In [2]:
import cv2
import numpy as np
import tensorflow as tf

1. 앞 단계와 동일하게 모델 로드 후 테스트 이미지를 넣고 추론을 실시합니다.

In [5]:
model = tf.saved_model.load("backup/best_saved_model")
src = cv2.imread("test_images/test_image.png")
src_converted = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)
src_resized = cv2.resize(src_converted, (640, 640))
src_resized = src_resized.reshape((1, 640, 640, 3))
src_resized = src_resized/255.
pred = model(src_resized)
pred = pred[0][0].numpy() # -> 추론결과를 담은 (25200, 7) 모양의 numpy array

2. 작업 용이성을 위해 cxcywh를 xyxy로 변환하는 함수를 미리 만들어 놓습니다.

In [4]:
def getXyxy(cxcywh):
    x1 = cxcywh[0] - (cxcywh[2]/2)
    y1 = cxcywh[1] - (cxcywh[3]/2)
    x2 = x1 + cxcywh[2]
    y2 = y1 + cxcywh[3]
    return [x1, y1, x2, y2]

3. 추론 결과를 입력으로 받은 후 obj conf.로 cutoff한 후(threshold), 클래스별로 결과를 분리해 출력해 주는 함수를 만들겠습니다.

In [ ]:
def getCandidatesByClass(pred, conf_thres=0.25): #pred : (25200, 7)
    #pred가 담고 있는 클래스의 개수가 몇개인지 계산해 class_len에 담아 놓는다.
    #pred[0] -> 25200개의 추론 결과 중 첫 번째 행
    #pred[0].shape -> (7,)
    #pred[0].shape[0] -> 7
    class_len = pred[0].shape[0] - 5 #5개 열(cx, cy, w, h, obj conf.)를 뺀 나머지 열이 클래스의 개수
    candidate_list = [] #바운딩박스 후보군을 담아놓을 빈 리스트를 하나 생성

    #클래스별로 바운딩박스 결과를 분리해 담기 위해, candidate_list에 클래스별 딕셔너리를 하나씩 추가
    for class_no in range(class_len):
        candidate_list.append({
            "class_no" : class_no, #클래스 번호 (0, 1)
            "bboxes(xyxy)" : [] #여기에 들어갈 값은 아래에서 추가할 것이므로 빈 리스트로 넣어 놓음
        })
    
    #25200 각 행으로 접근하여 conf_thres 값보다 높은 obj conf. 값을 가진 결과만 추려냄
    for item in pred:
        #클래스별 확률값을 담고 있는 요소들만 슬라이싱한 후 최대값이 위치한 곳의 index 번호 가져오기
        # -> 최대값이 위치한 곳의 index = class_no
        this_class_no = item[5:].argmax()
        this_conf = item[4]
        if this_conf >= conf_thres:
            #bbox의 너비 높이가 일정 수준 이하인 것들은 제거한다.
            bbox = getXyxy(item[:4])
            if bbox[2] - bbox[0] < min_wh or bbox[3] - bbox[1] < min_wh:
                continue
            this_result = bbox
            this_result.append(this_conf)
            candidate_list[this_class_no]["bboxes(xyxy)"].append(this_result)
    for class_no in range(class_len):
        if len(candidate_list[class_no]["bboxes(xyxy)"]) > 0:
            candidate_list[class_no]["bboxes(xyxy)"] = np.array(candidate_list[class_no]["bboxes(xyxy)"])
            candidate_list[class_no]["bboxes(xyxy)"] = candidate_list[class_no]["bboxes(xyxy)"][np.argsort(-candidate_list[class_no]["bboxes(xyxy)"][:, -1])]
    return candidate_list

In [40]:


def IoU(box1, box2):
    #서로 겹쳐져 있는지 확인하고, 겹쳐져 있지 않다면 0을 출력한다.
    if box1[2] < box2[0] or box1[0] > box2[2] or box1[3] < box2[1] or box1[1] > box2[3]:
        return 0

    # box = (x1, y1, x2, y2)
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
    
    # obtain x1, y1, x2, y2 of the intersection
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # compute the width and height of the intersection
    w = max(0, x2 - x1 + 1)
    h = max(0, y2 - y1 + 1)

    inter = w * h
    iou = inter / (box1_area + box2_area - inter)
    return iou

def nms(bboxes, iou_thres = 0.45, merge_thres = 0.75, print_iou=False): #겹치는 면적이 iou_thres를 넘으면 하나만 선택한다.
    if len(bboxes) == 0:
        return bboxes
    B = bboxes

    #1) 해당 클래스에 대해 최종적으로 남길 bounding box 를 담을 리스트 D를 생성
    # B 중에서 class score 가 가장 높은 bounding box 를 선택하고 D에 추가하고 B에서 삭제
    argmax_B = B[:, -1].argmax()
    T = B[argmax_B] #B의 최대값을 담을 임시 변수 T 생성
    B = np.delete(B, argmax_B, axis=0)
    D = None #검사가 완료된 최종 D만 담을 리스트

    #2) D에 추가된 class score 가 가장 높은 bounding box 와 B에 담긴 bounding box 와의 IOU (Intersection Over Union) 을 계산
    while True:
        idx_to_delete = []
        for idx, B_item in enumerate(B):
            # IOU 가 주어진 임계치 iou_thres보다 크다면 B에서 제거
            iou = IoU(T[:4], B_item[:4])
            # print(T[:4], B_item[:4])
            if iou > iou_thres:
                idx_to_delete.append(idx)
        B = np.delete(B, idx_to_delete, axis=0)
        if D is None:
            D = T.copy()
            D = D.reshape(1, 5)
        else:
            D = np.vstack([D, T])
        # print("D : {}".format(len(D)), "B : {}".format(len(B)))

        if len(B) > 0:
            #3) 2번 과정을 수행하고 B에 남은 bounding box 중 가장 큰 class score 를 가진 bounding box 를 D에 추가하고 B에서 제거한 후 2번 과정을 반복
            #B에 bounding box 가 남지 않을 때까지 반복
            argmax_B = B[:, -1].argmax()
            T = B[argmax_B]
            B = np.delete(B, argmax_B, axis=0)
        else:
            break
    return D

conf_thres = 0.25
min_wh = 0.002

candidate_list = getCandidatesByClass(pred, conf_thres, min_wh)

class_names = ["sign", "light"]
bbox_colors = [(0, 255, 0), (0, 0, 255)]

dst = cv2.resize(src, (int((900*src.shape[1])/src.shape[0]), 900), interpolation=cv2.INTER_LINEAR) #이미지가 너무 크므로 리사이즈한다.
for idx, candidate in enumerate(candidate_list):
    class_name = class_names[idx]
    color = bbox_colors[idx]
    bboxes = nms(candidate_list[idx]["bboxes(xyxy)"])
    for bbox in bboxes:
        conf = bbox[-1]

        x1 = int(bbox[0] * dst.shape[1])
        y1 = int(bbox[1] * dst.shape[0])
        x2 = int(bbox[2] * dst.shape[1])
        y2 = int(bbox[3] * dst.shape[0])

        dst = cv2.rectangle(dst, (x1, y1), (x2, y2), color, 2)
        dst = cv2.putText(dst, "{}:{:.2f}".format(class_name, conf), (x1, y1-5), cv2.FONT_HERSHEY_DUPLEX, 0.6, color, 1)
cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

카메라에 연결하여 테스트 해봅시다.

In [7]:
conf_thres = 0.25
min_wh = 0.002
class_names = ["sign", "light"]
bbox_colors = [(0, 255, 0), (0, 0, 255)]

# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture("test_images/test_video.mp4")

while True:
    retval, frame = cap.read()
    if retval is False:
        break
    src = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    src = cv2.resize(src, (640, 640))
    src = src/255.
    src = src.reshape((1, 640, 640, 3))
    pred = model(src)
    pred = pred[0][0].numpy()

    candidate_list = getCandidatesByClass(pred, conf_thres, min_wh)

    dst = frame.copy()
    for idx, candidate in enumerate(candidate_list):
        class_name = class_names[idx]
        color = bbox_colors[idx]
        bboxes = nms(candidate_list[idx]["bboxes(xyxy)"])
        for bbox in bboxes:
            conf = bbox[-1]

            x1 = int(bbox[0] * dst.shape[1])
            y1 = int(bbox[1] * dst.shape[0])
            x2 = int(bbox[2] * dst.shape[1])
            y2 = int(bbox[3] * dst.shape[0])

            dst = cv2.rectangle(dst, (x1, y1), (x2, y2), color, 2)
            dst = cv2.putText(dst, "{}:{:.2f}".format(class_name, conf), (x1, y1-5), cv2.FONT_HERSHEY_DUPLEX, 0.6, color, 1)
    cv2.imshow("dst", dst)
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()
cap.release()

Lessons

1. 물체감지는 라벨링 작업에 시간이 많이 걸리므로, 이번 실습은 일반적으로 공개된 데이터를 활용해 진행할 수 밖에 없었다.

2. 현업에서 실제 활용할 목적으로 모델을 만들고자 한다면, 수집한 이미지 데이터에 맞는 라벨링 작업이 필수로 요구된다.